In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [13]:
all_schools_url = 'https://www.sports-reference.com/cbb/seasons/men/2023-school-stats.html'
response = requests.get(all_schools_url, timeout = 10)
print(response.status_code)

team_name_id_dict = {}

soup = BeautifulSoup(response.content)
table = soup.findAll('tr')

for row in table:
    
    search = row.find('a',href = True)
    if search:
        url_suffix = str(search).split('"')[1].replace(".html","")
        team_name = str(search).split(">")[1].replace("</a","").strip()
        team_name_id_dict[team_name] = url_suffix

print(team_name_id_dict['Purdue'])

200
/cbb/schools/purdue/men/2023


In [17]:
master_df = pd.DataFrame()
counter = 0
stop_to_rest_point = np.random.randint(100,300)

url_prefix = 'https://www.sports-reference.com'

for team_name,url_suffix in tqdm(team_name_id_dict.items()):

    full_url = f"{url_prefix}{url_suffix}-gamelogs.html"
    temp_df = pd.read_html(full_url)[0]
    temp_df.columns = [col2 if (col1.startswith('Unnamed') or col1 == "School") else f"opp_{col2}" for col1,col2 in temp_df.columns]
    temp_df = temp_df.iloc[:,~temp_df.columns.str.startswith('Unnamed')].drop('G',axis = 1).dropna().query("Date != 'Date'")
    temp_df['team_name'] = team_name
    master_df = pd.concat([master_df,temp_df])
    counter +=1
    # if counter == stop_to_rest_point:
    #     time.sleep(np.random.randint(60,120))
    #     continue
    if counter == 3:
        break
    time.sleep(5)

  1%|          | 2/363 [00:13<41:49,  6.95s/it]


In [18]:
master_df

,G,Date,Unnamed: 2_level_1,Opp,W/L,Tm,Opp,FG,FGA,FG%,...,opp_FTA,opp_FT%,opp_ORB,opp_TRB,opp_AST,opp_STL,opp_BLK,opp_TOV,opp_PF,team_name
0,1,2022-11-07,NaN,Jackson State,W,65,56,23,57,.404,...,14,.714,7,40,9,6,1,21,21,Abilene Christian
1,2,2022-11-11,@,Texas A&M,L,58,77,20,52,.385,...,35,.714,10,33,7,11,3,19,17,Abilene Christian
2,3,2022-11-15,NaN,McMurry,W,104,46,41,68,.603,...,15,.667,4,17,7,9,2,27,18,Abilene Christian
3,4,2022-11-21,N,Wright State,L,61,77,25,58,.431,...,19,.632,1,24,18,12,5,18,14,Abilene Christian
4,5,2022-11-22,N,Weber State,L,67,77,26,53,.491,...,25,.920,8,29,10,4,0,18,13,Abilene Christian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,29,2023-02-25,NaN,Western Michigan,W,81,64,29,55,.527,...,10,.800,6,24,7,3,0,12,13,Akron
31,30,2023-02-28,NaN,Ball State,W,87,83,26,59,.441,...,11,.818,6,32,15,5,2,9,23,Akron
32,31,2023-03-03,@,Kent State,L (1 OT),84,89,25,57,.439,...,25,.880,7,33,13,7,4,13,25,Akron
33,32,2023-03-09,N,Buffalo,W,101,77,36,61,.590,...,25,.760,12,31,5,7,1,12,21,Akron
